# Texas Cosmetologist Violations

Texas has a system for [searching for license violations](https://www.tdlr.texas.gov/cimsfo/fosearch.asp). You're going to search for cosmetologists!

## Setup: Import what you'll need to scrape the page

We'll be using Selenium for this, *not* BeautifulSoup and requests.

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.firefox import GeckoDriverManager

/Users/yikchunlam/.pyenv/versions/3.10.3/lib/python3.10/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn(


## Starting your search

Starting from [here](https://www.tdlr.texas.gov/cimsfo/fosearch.asp), search for cosmetologist violations for people with the last name **Nguyen**.

In [2]:
driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())

/var/folders/86/xmjq1vh10t59mm3v94bs4l_r0000gn/T/ipykernel_76354/3328504527.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())


In [3]:
driver.get('https://www.tdlr.texas.gov/cimsfo/fosearch.asp')

In [6]:
driver.find_element(By.ID, 'pht_status').click()

In [7]:
driver.find_element(By.XPATH, '/html/body/div[1]/div/div[2]/div/div/section/div/div/table/tbody/tr/td/form/table/tbody/tr[3]/td/select/option[10]').click()

In [8]:
driver.find_element(By.ID, 'pht_lnm').send_keys('Nguyen')

In [9]:
driver.find_element(By.XPATH, '/html/body/div[1]/div/div[2]/div/div/section/div/div/table/tbody/tr/td/form/table/tbody/tr[18]/td/input[1]').click()

## Scraping

Once you are on the results page, do this. **I step you through things bit by bit, so it's going to be a little different than we did in class.** Also, no `pd.read_html` allowed because this isn't actual tabular data!

> You can use either Selenium by itself or Selenium+BeautifulSoup to scrape the results page. The choice is up to you!

### Loop through each result and print the entire row

Okay wait, maybe not, i's a heck of a lot of rows. Use `[:10]` to only do the first ten! For example, if you saved the table rows into `results` you might do something like this:

```python
for result in results[:10]:
    print(result)
```

Although you'd want to print out the text from the row (I give example output below).

> *Tip: If you're using Selenium, `By.TAG_NAME` is used if you don't have a class or ID. If you're using BeautifulSoup, just do your normal thing.*

In [22]:
rows = driver.find_elements(By.TAG_NAME, 'tr')

In [27]:
for row in rows[:10]:
    items = row.find_elements(By.TAG_NAME, 'td')
    for item in items:
        print(item.text)
    print("-----")

-----
NGUYEN, MAI T
City: AUSTIN
County: TRAVIS
Zip Code: 78717


License #: 750076

Complaint # COS20220000853
Date: 7/7/2022

Respondent is assessed an administrative penalty in the amount of $7,500.
Respondent failed to clean, disinfect, and sterilize metal instruments with a Department-approved sterilizer in accordance with the sterilizer or sanitizers manufacturer's instructions; Respondent failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used, the Department is charging 3 violations; Respondent failed to follow whirlpool foot spas cleaning and sanitization procedures as required, the Department is charging 3 violations; Respondent failed to clean and disinfect all wax pots; Respondent failed to clean diamond, carbide, natural and metal bits after each use with a brush or ultrasonic cleaner, or by immersing in acetone.
-----
NGUYEN, HUNG VU
City: HOUSTON
County: HARRIS
Zip Code: 77086


License #: 763912

Complai

The result should look something like this:

```
Name and Location Order Basis for Order
NGUYEN, THANH
City: FRISCO
County: COLLIN
Zip Code: 75034


License #: 790672

Complaint # COS20210004784 Date: 11/16/2021

Respondent is assessed an administrative penalty in the amount of $1,875. Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day, the Department is charging 2 violations; Respondent operated a cosmetology salon without the appropriate license.
NGUYEN, LONG D
City: SAN SABA
County: SAN SABA
Zip Code: 76877
```

### Loop through each result and print each person's name

You'll get an error because the first one doesn't have a name. How do you make that not happen?! If you want to ignore an error, you use code like this:

```python
try:
   # try to do something
except:
   print("It didn't work')
```

It should help you out. If you don't want to print anything when there's an error, you can type `pass` instead of the `print` statement.

**Why doesn't the first one have a name?**

Output should look like this:

```
Doesn't have a name
NGUYEN, THANH
NGUYEN, LONG D
NGUYEN, LUCIE HUONG
NGUYEN, CHINH
NGUYEN, JIMMY
```

* *Tip: The name has a class you can use. The class name is reused in a lot of places, but because it's the first one you don't have to worry about that!*
* *Tip: Instead of searching across the entire page – `driver.find_element` or `doc.select_one` – you should be doing your searching just inside of each **row** (I used this technique in the beginning of class with BeautifulSoup when we were scraping the books page)* 

In [30]:
for row in rows[:10]:
    try:
        name = row.find_element(By.CLASS_NAME, 'results_text')
        print(name.text)
    except:
        print("No Name")
    print("-----")

No Name
-----
NGUYEN, MAI T
-----
NGUYEN, HUNG VU
-----
NGUYEN, MINH HIEN LUONG
-----
NGUYEN, PHUONG T
-----
NGUYEN, STEVEN MANH
-----
NGUYEN, LANG THU
-----
NGUYEN, THANH
-----
NGUYEN, NAM QUANG
-----
NGUYEN, JAY TRI
-----


## Loop through each result, printing each violation description ("Basis for order")

Your results should look something like:

```
Doesn't have a violation
Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day, the Department is charging 2 violations; Respondent operated a cosmetology salon without the appropriate license.
Respondent failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used, the Department is charging 2 violations; Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements after each use; Respondent failed to clean and disinfect manicure tables prior to use for each client.
...
```

> - *Tip: You'll get an error even if you're ALMOST right - which row is causing the problem?*
> - *Tip: If you're using Selenium by itself, you can get the HTML of something by doing `.get_attribute('innerHTML')` – that way it'll look like BeautifulSoup when you print it. It might help you diagnose your issue!*
> - *Tip: Or I guess you could just skip the one with the problem...*

In [33]:
for row in rows[:10]:
    try:
        violation = row.find_elements(By.TAG_NAME, 'td')[-1]
        print(violation.text)
    except:
        print("Not a violation")
    print("-----")

Not a violation
-----
Respondent failed to clean, disinfect, and sterilize metal instruments with a Department-approved sterilizer in accordance with the sterilizer or sanitizers manufacturer's instructions; Respondent failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used, the Department is charging 3 violations; Respondent failed to follow whirlpool foot spas cleaning and sanitization procedures as required, the Department is charging 3 violations; Respondent failed to clean and disinfect all wax pots; Respondent failed to clean diamond, carbide, natural and metal bits after each use with a brush or ultrasonic cleaner, or by immersing in acetone.
-----
Respondent failed to keep floors, walls, ceilings, shelves, furniture, furnishings, and fixtures clean and in good repair; Respondent failed to clean and disinfect multiple use implements after each customer; Respondent failed to keep a record of the date and time of e

## Loop through each result, printing the complaint number

Output should look like this:

```
Doesn't have a complaint number
COS20210004784
COS20210009745
COS20210011484
...
```

- *Tip: Think about the order of the elements. Can you count from the opposite direction than you normally do?*

In [39]:
for row in rows[:10]:
    try:
        complaint_no = row.find_elements(By.CLASS_NAME, 'results_text')[-2]
        print(complaint_no.text)
    except:
        print("Doesn't Have A Complaint #")
    print("-----")

Doesn't Have A Complaint #
-----
COS20220000853
-----
COS20200013887
-----
COS20210010437
-----
COS20210014779
-----
COS20200015190
-----
COS20200008154
-----
COS20200007790
-----
COS20210010525
-----
COS20210015763
-----


## Saving the results

### Loop through each result to create a list of dictionaries

Each dictionary must contain

- Person's name
- Violation description
- Violation number
- License Numbers
- Zip Code
- County
- City

Create a new dictionary for each result (except the header).

Based on what you print out, the output might look something like:

```
This row is broken: Name and Location Order Basis for Order
{'name': 'NGUYEN, THANH', 'city': 'FRISCO', 'county': 'COLLIN', 'zip_code': '75034', 'complaint_no': 'COS20210004784', 'license_numbers': '790672', 'complaint': 'Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day, the Department is charging 2 violations; Respondent operated a cosmetology salon without the appropriate license.'}
{'name': 'NGUYEN, LONG D', 'city': 'SAN SABA', 'county': 'SAN SABA', 'zip_code': '76877', 'complaint_no': 'COS20210009745', 'license_numbers': '760420, 1620583', 'complaint': 'Respondent failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used, the Department is charging 2 violations; Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements after each use; Respondent failed to clean and disinfect manicure tables prior to use for each client.'}
```

> *Tip: If you want to ask for the "next sibling," you can't use `find_next_sibling` in Selenium, you need to use `element.find_element_by_xpath("following-sibling::div")` to find the next div, or `element.find_element_by_xpath("following-sibling::*")` to find the next anything.

In [132]:
violators = []
for row in rows[:10]:
    try:
        record = {}
        items = row.find_elements(By.TAG_NAME, 'td')
        complaint = items[-1].text
        for item in items:
            section = item.find_elements(By.CLASS_NAME, 'results_text')
            record['name'] = section[0].text
            record['city'] = section[1].text
            record['county'] = section[2].text
            record['zip_code'] = section[3].text
            record['complaint_no'] = section[4].text
            record['license_numbers'] = section[5].text
            record['complaint'] = complaint
            violators.append(record)
    except:
        continue

[{'name': '7/7/2022',
  'city': 'AUSTIN',
  'county': 'TRAVIS',
  'zip_code': '78717',
  'complaint_no': '750076',
  'license_numbers': 'COS20220000853',
  'complaint': "Respondent failed to clean, disinfect, and sterilize metal instruments with a Department-approved sterilizer in accordance with the sterilizer or sanitizers manufacturer's instructions; Respondent failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used, the Department is charging 3 violations; Respondent failed to follow whirlpool foot spas cleaning and sanitization procedures as required, the Department is charging 3 violations; Respondent failed to clean and disinfect all wax pots; Respondent failed to clean diamond, carbide, natural and metal bits after each use with a brush or ultrasonic cleaner, or by immersing in acetone."},
 {'name': '7/6/2022',
  'city': 'HOUSTON',
  'county': 'HARRIS',
  'zip_code': '77086',
  'complaint_no': '763912',
  'licen

### Save that to a CSV named `output.csv`

The dataframe should look something like...

|index|name|city|county|zip_code|complaint_no|license_numbers|complaint|
|---|---|---|---|---|---|---|---|
|0|NGUYEN, THANH|FRISCO|COLLIN|75034|COS20210004784|790672|Respondent failed to clean and sanitize whirlp...|
|1|NGUYEN, LONG D|SAN SABA|SAN SABA|76877|COS20210009745|760420, 1620583|Respondent failed to keep a record of the date...|


- *Tip: If you send a list of dictionaries to `pd.DataFrame(...)`, it will create a dataframe out of that list!*

In [106]:
import pandas as pd
pd.DataFrame(violators)

,name,city,county,zip_code,complaint_no,license_numbers,complaint
0,7/7/2022,AUSTIN,TRAVIS,78717,750076,COS20220000853,"Respondent failed to clean, disinfect, and ste..."
1,7/6/2022,HOUSTON,HARRIS,77086,763912,COS20200013887,"Respondent failed to keep floors, walls, ceili..."
2,6/27/2022,CARROLLTON,DALLAS,75007,777932,COS20210010437,Respondent failed to clean and sanitize whirlp...
3,6/16/2022,PLANO,COLLIN,75023,766293,COS20210014779,Respondent failed to keep a record of the date...
4,6/9/2022,FORT WORTH,TARRANT,76244,773318,COS20200015190,Respondent leased space in a salon to an indiv...
5,6/2/2022,ALLEN,COLLIN,75002,744463,COS20200008154,Respondent failed to clean and sanitize whirlp...
6,5/2/2022,FRISCO,COLLIN,75034,790672,COS20200007790,Respondent failed to follow whirlpool foot spa...
7,4/27/2022,HARKER HEIGHTS,BELL,76548,745552,COS20210010525,Respondent failed to keep a record of the date...
8,4/27/2022,SAN ANTONIO,BEXAR,78266,735165,COS20210015763,"Respondent failed to clean, disinfect, and ste..."


### Open the CSV file and examine the first few. Make sure you didn't save an extra weird unnamed column.